<a href="https://colab.research.google.com/github/girinath18/CodeFormer/blob/master/face_enhanacement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from IPython.display import clear_output
%cd /content
!rm -rf /content/CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer
!pip install -r /content/CodeFormer/requirements.txt
# Install basicsr
!python /content/CodeFormer/basicsr/setup.py develop
# Download the pre-trained model
!python /content/CodeFormer/scripts/download_pretrained_models.py facelib
!python /content/CodeFormer/scripts/download_pretrained_models.py CodeFormer
clear_output()

In [12]:
from google.colab import drive
import os
drive.mount("/content/gdrive", force_remount=True)
drive_input_folder = "/content/gdrive/MyDrive/upload"
if os.path.exists(drive_input_folder):
    print("The folder exists.")
else:
    os.mkdir(drive_input_folder)
    print(f"Creating {drive_input_folder} folder")

Mounted at /content/gdrive
The folder exists.


In [17]:
#@title Utils { display-mode: "form" }


# Visualization function
import cv2
import matplotlib.pyplot as plt
import os
import shutil
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

upload_folder = ('/content/input')
if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)
if os.path.exists("/content/output"):
  shutil.rmtree("/content/output")
clear_output()

In [18]:
from PIL import Image
import os
import shutil
from google.colab import drive
import os
from google.colab import files
keep_images = False
upload_folder = ('/content/input')
%cd /content/CodeFormer

if keep_images:
  pass
else:
  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
  os.mkdir(upload_folder)


def image_from_drive():
  drive.mount('/content/gdrive',force_remount=True)
  drive_input_folder = "/content/input"
  if os.path.exists(drive_input_folder):
      print("The folder exists.")
  else:
      os.mkdir(drive_input_folder)
  image_extensions = ['.jpg', '.jpeg', '.png']
  for filename in os.listdir(drive_input_folder):
    _, extension = os.path.splitext(filename)
    if extension.lower() in image_extensions:
      drive_image_path=os.path.join(drive_input_folder,filename)
      shutil.copy(drive_image_path,upload_folder)
def image_from_device():
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(upload_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)

upload_from = "local device"
if upload_from == "google drive":
  image_from_drive()
if upload_from == "local device":
  image_from_device()
clear_output()
file_count = len(os.listdir(upload_folder))
if file_count >= 1:
    print("Run next cell")
else:
    print("Please upload an image.")


Run next cell


In [19]:
# Inference the uploaded images
%cd /content/CodeFormer

CODEFORMER_FIDELITY = 0.7

BACKGROUND_ENHANCE = True

FACE_UPSAMPLE = True
if BACKGROUND_ENHANCE:
  if FACE_UPSAMPLE:
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload --bg_upsampler realesrgan --face_upsample
  else:
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload --bg_upsampler realesrgan
else:
  !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload
clear_output()
print(f"All results are saved in /content/output{CODEFORMER_FIDELITY}")

All results are saved in /content/output0.7


In [5]:

save_a_copy_in_google_drive = True
if save_a_copy_in_google_drive:
  drive.mount('/content/gdrive',force_remount=True)
  drive_save_path = '/content/gdrive/MyDrive/CodeFormer_Bulk_Upscale'
  if os.path.exists(drive_save_path):
    pass
  else:
    os.mkdir(drive_save_path)
  clear_output()
  print(f"All images/zip file save at : {drive_save_path}")

import os
import uuid
from google.colab import files
import shutil
def download_single_images():
  global CODEFORMER_FIDELITY
  download_folder = '/content/download'
  if os.path.exists(download_folder):
      os.system(f'rm -rf {download_folder}')
  os.makedirs(download_folder)
  folder_path = f"/content/CodeFormer/results/user_upload_{CODEFORMER_FIDELITY}/final_results"
  for filename in os.listdir(folder_path):
      original_path = os.path.join(folder_path, filename)
      name, extension = os.path.splitext(filename)
      random_string = str(uuid.uuid4())[:8]
      new_filename = f"{name}_{random_string}{extension}"
      download_path = f"/content/download/{new_filename}"
      if save_a_copy_in_google_drive:
        drive_path=f"{drive_save_path}/{new_filename}"
        shutil.copy(original_path, drive_path)
      shutil.copy(original_path, download_path)

      files.download(download_path)
def download_zip():
  global CODEFORMER_FIDELITY
  random_string = str(uuid.uuid4())[:5]
  zip_file_name=f"results_{random_string}.zip"
  var1=os.system(f'zip -r {zip_file_name} results/user_upload_{CODEFORMER_FIDELITY}/final_results')
  if save_a_copy_in_google_drive:
    sour=f"/content/CodeFormer/{zip_file_name}"
    dest=f"{drive_save_path}/{zip_file_name}"
    shutil.copy(sour, dest)

  files.download(zip_file_name)



folder_path = f"/content/CodeFormer/results/user_upload_{CODEFORMER_FIDELITY}/final_results"
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']  # Add more extensions if needed
contains_image = False
for filename in os.listdir(folder_path):
    _, extension = os.path.splitext(filename)
    if extension.lower() in image_extensions:
        contains_image = True
        break
download_format = "zip"
  if download_format == "single file":
    download_single_images()
  if download_format == "zip":
    download_zip()
else:
  print(f"The folder '{folder_path}' does not contain any image files.")


IndentationError: unexpected indent (<ipython-input-5-10595a1c01dd>, line 58)

# **maded as the single script**

In [ ]:
# Required Libraries
import os
import shutil
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# Visualization Function
def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.title('Input', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('CodeFormer', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

def imread(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image file not found at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Paths
input_folder = '/content/input'
output_folder = '/content/output_bots'
codeformer_path = '/content/CodeFormer'

# Clean up previous runs
if os.path.isdir(input_folder):
    shutil.rmtree(input_folder)
os.mkdir(input_folder)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

# Image Upload
def image_from_device():
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(input_folder, filename)
        print(f'Move {filename} to {dst_path}')
        shutil.move(filename, dst_path)

# Execute Image Upload
image_from_device()
clear_output()
file_count = len(os.listdir(input_folder))
if file_count >= 1:
    print("Run next cell")
else:
    print("Please upload an image.")

# Inference the uploaded images
os.chdir(codeformer_path)

CODEFORMER_FIDELITY = 0.7
BACKGROUND_ENHANCE = True
FACE_UPSAMPLE = True

# Running the inference and saving outputs in specified output folder
command = f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path {input_folder} --output_path {output_folder}'
if BACKGROUND_ENHANCE:
    command += ' --bg_upsampler realesrgan'
if FACE_UPSAMPLE:
    command += ' --face_upsample'

os.system(command)

# Debugging information to verify the output
print(f"All results are saved in {output_folder}")
output_files = os.listdir(output_folder)
print(f"Files in output folder: {output_files}")

# Check if output folder contains any images
if not output_files:
    raise FileNotFoundError(f"No output images found in {output_folder}")

# Display Results
input_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
output_image_path = os.path.join(output_folder, output_files[0])

print(f"Input image path: {input_image_path}")
print(f"Output image path: {output_image_path}")

input_image = imread(input_image_path)
output_image = imread(output_image_path)
display(input_image, output_image)


In [ ]:
# Required Libraries
import os
import shutil
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# Visualization Function
def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.title('Input', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('CodeFormer', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

def imread(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"Image file not found at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

# Paths
input_folder = '/content/input'
output_folder = '/content/output_bots'
codeformer_path = '/content/CodeFormer'

# Clean up previous runs
if os.path.isdir(input_folder):
    shutil.rmtree(input_folder)
os.mkdir(input_folder)
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.mkdir(output_folder)

# Image Upload
def image_from_device():
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(input_folder, filename)
        print(f'Move {filename} to {dst_path}')
        shutil.move(filename, dst_path)

# Execute Image Upload
image_from_device()
clear_output()
file_count = len(os.listdir(input_folder))
if file_count >= 1:
    print("Run next cell")
else:
    print("Please upload an image.")

# Inference the uploaded images
os.chdir(codeformer_path)

CODEFORMER_FIDELITY = 0.7
BACKGROUND_ENHANCE = True
FACE_UPSAMPLE = True

# Running the inference and saving outputs in specified output folder
command = f'python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path {input_folder} --output_path {output_folder}'
if BACKGROUND_ENHANCE:
    command += ' --bg_upsampler realesrgan'
if FACE_UPSAMPLE:
    command += ' --face_upsample'

os.system(command)

# Debugging information to verify the output
print(f"All results are saved in {output_folder}")
output_files = os.listdir(output_folder)
print(f"Files in output folder: {output_files}")

# Check if output folder contains any images
final_results_folder = os.path.join(output_folder, 'final_results')
if not os.path.exists(final_results_folder):
    raise FileNotFoundError(f"No final_results folder found in {output_folder}")

final_results_files = os.listdir(final_results_folder)
if not final_results_files:
    raise FileNotFoundError(f"No output images found in {final_results_folder}")

# Display Results
input_image_path = os.path.join(input_folder, os.listdir(input_folder)[0])
output_image_path = os.path.join(final_results_folder, final_results_files[0])

print(f"Input image path: {input_image_path}")
print(f"Output image path: {output_image_path}")

input_image = imread(input_image_path)
output_image = imread(output_image_path)
display(input_image, output_image)
